In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import requests
import pandas as pd
from datetime import datetime, date
import time
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')
import os
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import json
import re
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
import nltk
from datetime import datetime, date
import sys
from better_profanity import profanity
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from pathlib import Path
import os
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
import re
from textblob import TextBlob
import spacy
from spacy import displacy

In [ ]:
def convert_df(df):
   return df.to_csv(index=False).encode('utf-8')

In [ ]:
def introduction():
    st.title("Analyzing Dreams using NLP")

In [ ]:
def data_collection():
    col1,col2,col3,col4 = st.columns()
    with col1:
        client_id = st.text_input("Reddit Client Id")
    with col2:
        secret_key = st.text_input("Reddit Secret Key")
    with col3:
        username = st.text_input("Reddit User Name")
    with col4:
        password = st.text_input("Reddit Password")
    time_wanted = datetime(2023, 1, 20, 00, 00, 00, 342380)

    client_id = client_id
    secret_key = secret_key

    auth = requests.auth.HTTPBasicAuth(client_id, secret_key)

    data = {
        'grant_type': 'password',
        'username': username,
        'password': password
    }

    headers = {'User-Agent': 'MyAPI/0.0.1'}

    res = requests.post('https://www.reddit.com/api/v1/access_token', 
                        auth = auth, 
                        data = data,
                        headers = headers)

    token = res.json()['access_token']

    headers['Authorization'] = f'bearer {token}'    

    def reddit_data(time_wanted):
        stopper = 0

        #initial set collection
        res = requests.get('https://oauth.reddit.com/r/Dreams/new',
                        headers = headers, params={'limit': '100'})
        
        df = pd.DataFrame()

        for post in res.json()['data']['children']:
            df = df.append({
                'subreddit': post['data']['subreddit'],
                'title': post['data']['title'],
                'text': post['data']['selftext'],
                'date': post['data']['created']
            }, ignore_index = True)
        
        #further back collection
        latest_key = post['kind'] + '_' + post['data']['id']

        while df.tail(1)['date'][df.tail(1)['date'].index[0]] > datetime.timestamp(time_wanted):
            for req in range(100):
            
                res = requests.get('https://oauth.reddit.com/r/Dreams/new',
                                    headers = headers, 
                                    params={'limit': '100', 'after': latest_key})
                
                for post in res.json()['data']['children']:
                    df = df.append({
                        'subreddit': post['data']['subreddit'],
                        'title': post['data']['title'],
                        'text': post['data']['selftext'],
                        'date': post['data']['created']
                    }, ignore_index = True)

                latest_key = post['kind'] + '_' + post['data']['id']
                    
                print(f'{len(df)} rows collected')

                if len(df) >= 988:
                    latest = df.tail(1)['date'][df.tail(1)['date'].index[0]]
                    print("Data Collection Target Reached")
                    print(f'{len(df)} rows collected')
                    print(f'latest subreddit date: {datetime.fromtimestamp(latest)}')
                    return df

                # time.sleep(5)
        else: 
            print("Date Limit Reached")
            print(f'{len(df)} rows collected')
            return df

    df = reddit_data(time_wanted)

    csv = convert_df(df)

    st.download_button(
    "Press to Download",
    csv,
    "file.csv",
    "text/csv",
    key='download-csv'

    )

In [ ]:
def data_cleaning(df):
    nltk.download('stopwords')
    nltk.download('omw-1.4')
    nltk.download('wordnet')
    stopword = nltk.corpus.stopwords.words('english')
    wn = nltk.WordNetLemmatizer()

    df = pd.read_csv(st.file_uploader(label = "Upload your Raw Data CSV"), index_col= 0)

    df= df.dropna()
    df['date'] = [datetime.fromtimestamp(time) for time in df['date']]
    df['text'] = [profanity.censor(i) for i in df['text']]

    #calculating length of each dream
    df['length'] = [len(j) for j in df['text']]

    # if less than or equal to 5th percentile, assign t_f column False
    df['t_f'] = [True if j > np.percentile(df['length'], 5) else False for j in df['length']]

    #only keep t_f == True rows
    semi = df.loc[df['t_f'] == True, :].__deepcopy__()

    def clean(text):
        text = re.sub('https?://\S+|www\.\S+', '', text) #replace website urls
        text = re.sub(r"@\S+", '', text) #replace anything that follows @
        text = re.sub(r"#\S+", '', text) #replace anything that follows #
        text = re.sub(r"[0-9]", '', text) #replace numeric
        text = re.sub(r"\n", '', text) #replace new line 
        text = re.sub("\'m", ' am ', text) 
        text = re.sub("\'re", ' are ', text) 
        text = re.sub("\'d", ' had ', text)
        text = re.sub("\'s", ' is ', text)
        text = re.sub("\'ve", ' have ', text)
        text = re.sub(" im ", ' i am ', text)
        text = re.sub(" iam ", ' i am ', text)
        text = re.sub(" youre ", ' you are ', text)
        text = re.sub(" theyre ", ' they are ', text)
        text = re.sub(" theyve ", ' they have ', text)
        text = re.sub(" weve ", ' we have ', text)
        text = re.sub(" isnt ", ' is not ', text)
        text = re.sub(" arent ", ' are not ', text)
        text = re.sub(" ur ", ' you are ', text)
        text = re.sub(" ive ", ' i have ', text)
        text = re.sub("_", '', text)
        text = re.sub("\"", '', text)
        text = re.sub(" bc ", ' because ', text)
        text = re.sub(" aka ", ' also known as ', text)
        text = re.sub("√©", 'e', text) #encoding error for é. replace it with e
        text = re.sub(" bf  ", ' boyfriend ', text)
        text = re.sub(" gf  ", ' girlfriend ', text)
        text = re.sub(" btw  ", ' by the way ', text)
        text = re.sub(" btwn  ", ' between ', text)
        text = re.sub(r'([a-z])\1{2,}', r'\1', text) #if the same character is repeated more than twice, remove it to one. (E.A. ahhhhhh --> ah)
        text = re.sub(' ctrl ', ' control ', text)
        text = re.sub(' cuz ', ' because ', text)
        text = re.sub(' dif ', ' different ', text)
        text = re.sub(' dm ', ' direct message ', text)
        text = re.sub("n't", r' not ', text)
        text = re.sub(" fav ", ' favorite ', text)
        text = re.sub(" fave ", ' favorite ', text)
        text = re.sub(" fml ", " fuck my life ", text)
        text = re.sub(" hq ", " headquarter ", text)
        text = re.sub(" hr ", " hours ", text)
        text = re.sub(" idk ",  "i do not know ", text)
        text = re.sub(" ik ", ' i know ', text)
        text = re.sub(" lol ", ' laugh out loud ', text)
        text = re.sub(" u ", ' you ', text)
        text = re.sub("√¶", 'ae', text) #encoding error for áe. replace it with ae
        text = re.sub("√® ", 'e', text) #encoding error for é. replace it with e
        text = text.strip()
        return text

    def tokenization(text):
        text = re.split('\W+', text) #split words by whitespace to tokenize words
        return text

    def remove_stopwords(text):
        text = [word for word in text if word not in stopword] #remove stopwords in the nltk stopwords dictionary
        return text

    def lemmatizer(text):
        text = [wn.lemmatize(word) for word in text] #lemmatize the tokenized words. Lemmatized > Stemming in this case
        return text                                  #because lemmatizing keeps the context of words alive

    def vectorization(li):                            #create matrix of words and its respective presence for each dream
        vectorizer = CountVectorizer()   
        Xs = vectorizer.fit_transform(li)   
        X = np.array(Xs.todense())
        
        return X

    def get_column_name(li):                          #extract each word so that it will be present in corpus as column names
        vectorizer = CountVectorizer()   
        Xs = vectorizer.fit_transform(li)   
        col_names=vectorizer.get_feature_names_out()
        col_names = list(col_names)

        return col_names

    def extract_array(df):
        clean_text = df['text'].apply(lambda x:clean(x.lower()))         #first clean the text on lower cased list of dreams
        tokenized = clean_text.apply(lambda x: tokenization(x))          #tokenize the cleaned text
        clean_text = tokenized.apply(lambda x: " ".join(x))              #rejoin the words (just in case white space still present)
        print("Complete: text cleaning")
        print("Complete: tokenization")
        x_stopwords = tokenized.apply(lambda x: remove_stopwords(x))     #remove stopwords from tokenized list
        print("Complete: stopwords removed")
        lemmatized = x_stopwords.apply(lambda x: lemmatizer(x))          #lemmatize the removed stopwords word list
        print("Complete: lemmatization")
        complete = lemmatized.apply(lambda x: " ".join(x))               #rejoin the words so it will look like a sentence
        mapx = vectorization(complete)                                   #start of mapping to corpus
        name = get_column_name(complete)
        mapx = pd.DataFrame(mapx, columns = name)
        mapx.columns = name
        print("Complete: vectorization")
        print("All Done!")

    clean_text, tokenized, x_stopwords, lemmatized, complete, corpus = extract_array(semi)

    titles = ['clean_text', 'tokenized', 'x_stopwords', 'lemmatized', 'complete', 'corpus']

    st.write("Download Clean Datasets")
    col1,col2,col3,col4,col5,col6,col7 = st.columns([1,1,1,1,1,1,1])

    for k,context in enumerate(titles):
        x = pd.DataFrame({'title': semi['title'],
            context: vars()[context]}).reset_index()
        x = x.drop("index", axis =1)

        vars()[f'{context}_csv'] = convert_df(x)

        with vars()[f'col{k+1}']:
            st.download_button(
            f"{context}",
            vars()[f'{context}_csv'],
            f"{context}.csv",
            "text/csv",
            key='download-csv'

    )


In [ ]:
@st.cache_data
def data_visualization_1():
    nlp = spacy.load("en_core_web_sm")

    try:
        uploaded = st.file_uploader(label = "Upload your Complete Data CSV")
        df = pd.read_csv(uploaded, index_col= 0)

        
        tag_dict = {"word" :[], "tag":[]}
        full = []

        for i in df['complete']:
            sent = nlp(i)
            for j in sent:
                tag_dict['word'].append(j.text)
                tag_dict['tag'].append(j.tag_)

        tag_df  = pd.DataFrame(tag_dict)

        def barplot(x, z="", l = False):
            t = np.unique(x, return_counts = True)
            s = np.argsort(t[1])

            if l == True:
                x = t[0][s][-z:]
                y = t[1][s][-z:]
            else:   
                x = t[0][s]
                y = t[1][s]

            fig = plt.figure(figsize=(8,8)) #INITIALIZE FIGURE 
            ax = fig.add_subplot()

            ax.bar(x,y)

            ax.set_title(f"Barplot", fontsize = 20)
            ax.set_xlabel(f"", fontsize = 15)
            ax.set_ylabel("Count", fontsize = 15)
            plt.xticks(rotation=90)
            st.pyplot(fig)    

        barplot(tag_df['tag'])

        def wordcloud(x, lim):
            text = " ".join(x)
            cloud = WordCloud(collocations = False, max_words = lim).generate(text)
            plt.imshow(cloud, interpolation='bilinear')
            plt.axis("off")
            plt.show()
            st.pyplot()

        with st.container():
            tab1, tab2, tab3 = st.tabs(np.unique(tag_df['tag'])[:3])

            for i in np.unique(tag_df['tag']):
                if i <3:
                    with vars()[f'tab{i+1}']:
                        temp = list(tag_df.loc[tag_df['tag'] == i , 'word'])
                        print(i)
                        #print(i, tag_desc[i])
                        try:
                            wordcloud(temp, lim = 100)
                        except ValueError:
                            print("Word Cloud ValueError \n")
                            pass
                else: break

    except ValueError:
        st.write("Please Upload the CSV file")    







In [ ]:
def part_of_speech_tag():
    st.title("Part of Speech Tagging Visualization")

    nlp = spacy.load("en_core_web_sm")

    try:
        uploaded = st.file_uploader(label = "Upload your Semi Raw Data CSV")
        df = pd.read_csv(uploaded, index_col= 0)   

        row_n = int(st.text_input("Which Dream Would You Like to Examine? (In Row Number)"))
        sentence = list(nlp(df['text'][row_n]).sents) 
        rend = displacy.render(sentence, style = "dep", jupyter=False, options = {'compact': True})

        st.image(rend, width=400, use_column_width='never')

    except ValueError:
        st.write("Please Upload the CSV file")    

In [ ]:
def lda():
    #only caveat will be that ldavisualization cannot be embedded into the document, but we will need to prompt the users to go outside the page
    print(None)

In [ ]:
def tf_idf():
    print(None)

In [ ]:
page_names_to_funcs = {
    "Introduction": introduction,
    "Data Collection": data_collection,
    "Data Cleaning": data_cleaning,
    "Data Visualization": data_visualization_1,
    "Part of Speech Tagging": part_of_speech_tag,
    "Name Identity Recognition": name_identity_recognition,
    "Latency Discriminant Analysis (LDA)": lda,
    "TF-IDF": tf_idf,
}

demo_name = st.sidebar.selectbox("Please Select a Page", page_names_to_funcs.keys())
page_names_to_funcs[demo_name]()